# Produce random files for given nuclide and library

In [1]:
import sandy

In this examples we'll produce random ACE files for the JEFF-3.3 Pu-238 evaluation.
This sequence can we further extended for other nuclides, individually or together in a loop.

In [2]:
lib_name = "jeff_33"
nucl = 942390
filename = f"{nucl}.{lib_name}"

In [3]:
endf6 = sandy.get_endf6_file(lib_name, 'xs', nucl)
endf6.to_file(filename)

In [4]:
nsmp = 2  # number of samples
processes = 1  # change if you want to run in parallel (better on Linux)

In [5]:
# command line
cli = f"{filename}  --processes {processes}  --samples {nsmp}  --mf 33  --temperatures 300  --acer  --debug"
sandy.sampling.run(cli.split())

INFO:  processing file: '942390.jeff_33'
INFO:  Zero or no temperature was requested, NJOY processing will stop after RECONR.
    If you want to process 0K cross sections use `temperature=0.1`.
    


moder
20 -21 /
reconr
-21 -22 /
'sandy runs njoy'/
9437 0 0 /
1.0 0. /
0/
moder
-22 30 /
errorr
-21 -22 0 33 0 /
9437 2 2 0 1 /
0 0.0 /
0 33 0/
stop

 njoy 2016.72  29Sep23                                       10/11/23 16:33:02
 *****************************************************************************

 moder...                                                                 0.0s

 reconr...                                                                0.1s

 ---message from rdf2bw---calculation of angular distribution not installed.

 moder...                                                                 3.0s

 errorr...                                                                3.5s

 processing mat  9437
 ---------------------
  94-Pu-239 IRSN-CEA   EVAL-FEV16 IRSN-CEA DAM/DEN COLLAB.           

 covariances calculated for  8 reactions and  248 groups                  4.0s
                                                                          6.9s
 ******************

INFO:  writing to file 'PERT_94239_MF33.xlsx'...
INFO:  Zero or no temperature was requested, NJOY processing will stop after RECONR.
    If you want to process 0K cross sections use `temperature=0.1`.
    


moder
20 -21 /
reconr
-21 -22 /
'sandy runs njoy'/
9437 0 0 /
1.0 0. /
0/
moder
-22 30 /
stop

 njoy 2016.72  29Sep23                                       10/11/23 16:33:15
 *****************************************************************************

 moder...                                                                 0.0s

 reconr...                                                                0.1s

 ---message from rdf2bw---calculation of angular distribution not installed.

 moder...                                                                 3.1s
                                                                          3.5s
 *****************************************************************************
Processing xs sample 0...
moder
20 -21 /
moder
99 -22 /
broadr
-21 -22 -23 /
9437 1 0 0 0. /
1.0 /
300.0 /
0 /
moder
-23 30 /
acer
-21 -23 0 50 70 /
1 0 1 .03 0 /
'sandy runs acer'/
9437 300.0 /
1 1 /
/
stop

 njoy 2016.72  29Sep23                                       1

INFO:  Total running time: 27.15 sec


                                                                          2.7s
 *****************************************************************************
writing to file '94239_1.03c'
writing to file '94239_1.03c.xsd'


This is equivalent to the command line entry
```bash
python -m sandy.sampling  942390.jeff_33  --processes 1  --samples 2  --temperatures 300  --acer  --debug
```

> Notice that option `--debug` should be removed for production.

If you need more control over your pipeline you can generate samples using the python API.

Some of the reasons identified to do this are:
- have access to the perturbation coefficients
- change the parameter `err` (used for by the NJOY modules RECONR and BROADR) to make the code run faster
- remove some NJOY modules such as PURR
- apply perturbations to already Doppler-broadenened cross sections (more details below)

This is equivalent to what was previously run.

In [6]:
# this generates samples for cross sections and nubar
smps = endf6.get_perturbations(
    nsmp,
    njoy_kws=dict(
        err=1,   # very fast calculation, for testing
        chi=False,
        mubar=False,
        xs=True,
        nubar=False,
        verbose=True,
    ),
)

INFO:  Zero or no temperature was requested, NJOY processing will stop after RECONR.
    If you want to process 0K cross sections use `temperature=0.1`.
    


moder
20 -21 /
reconr
-21 -22 /
'sandy runs njoy'/
9437 0 0 /
1.0 0. /
0/
moder
-22 30 /
errorr
-21 -22 0 33 0 /
9437 2 2 0 1 /
0 0.0 /
0 33 0/
stop

 njoy 2016.72  29Sep23                                       10/11/23 16:33:29
 *****************************************************************************

 moder...                                                                 0.0s

 reconr...                                                                0.1s

 ---message from rdf2bw---calculation of angular distribution not installed.

 moder...                                                                 3.1s

 errorr...                                                                3.5s

 processing mat  9437
 ---------------------
  94-Pu-239 IRSN-CEA   EVAL-FEV16 IRSN-CEA DAM/DEN COLLAB.           

 covariances calculated for  8 reactions and  248 groups                  4.0s
                                                                          6.9s
 ******************

INFO:  writing to file 'PERT_94239_MF33.xlsx'...


In [7]:
# This part applies the perturbations (samples) to the best-estimates and writes perturbed output files samples for cross sections and nubar
outs = endf6.apply_perturbations(
    smps,
    processes=processes,
    njoy_kws=dict(err=1),   # very fast calculation, for testing
    to_ace=True,   # produce ACE files
    to_file=True,
    ace_kws=dict(err=1, temperature=300, verbose=True, purr=False, heatr=False, thermr=False, gaspr=False),
    verbose=True,
)

INFO:  Zero or no temperature was requested, NJOY processing will stop after RECONR.
    If you want to process 0K cross sections use `temperature=0.1`.
    



 njoy 2016.72  29Sep23                                       10/11/23 16:33:41
 *****************************************************************************

 moder...                                                                 0.0s

 reconr...                                                                0.1s

 ---message from rdf2bw---calculation of angular distribution not installed.

 moder...                                                                 3.0s
                                                                          3.5s
 *****************************************************************************
Processing xs sample 0...
moder
20 -21 /
moder
99 -22 /
broadr
-21 -22 -23 /
9437 1 0 0 0. /
1.0 /
300.0 /
0 /
moder
-23 30 /
acer
-21 -23 0 50 70 /
1 0 1 .03 0 /
'sandy runs acer'/
9437 300.0 /
1 1 /
/
stop

 njoy 2016.72  29Sep23                                       10/11/23 16:33:47
 ****************************************************************************